%tensorflow_version 2.x

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf

import keras
import cv2
from keras.datasets import cifar10

Using TensorFlow backend.


In [2]:
batch_size = 32 
num_classes = 10 
epochs = 10
learning_rate = 0.001
input_shape = (32, 32, 3)

In [3]:
# загрузка данных
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print(y_train.shape, 'y_train samples')
print(y_test.shape, 'y_test samples')

x_train shape: (50000, 32, 32, 3)
x_test shape: (10000, 32, 32, 3)
(50000, 1) y_train samples
(10000, 1) y_test samples


In [4]:
# Нормализаци данных
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

y_train = y_train.reshape(-1).astype(np.int32)
y_test = y_test.reshape(-1).astype(np.int32)

print( x_train.shape)
print(y_train.shape)

(50000, 32, 32, 3)
(50000,)


In [5]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds = train_ds.shuffle(buffer_size=x_train.shape[0])
train_ds = train_ds.repeat(epochs)
train_ds = train_ds.batch(batch_size)

In [8]:
class Model(tf.keras.Model):
    
    def __init__(self):
        super(Model, self).__init__()
        
        self.conv1 = tf.keras.layers.Conv2D(32, (5, 5), activation='relu', padding='same')
        self.conv2 = tf.keras.layers.Conv2D(64, (5, 5), activation='relu', padding='same')
        self.conv3 = tf.keras.layers.Conv2D(128, (5, 5), activation='relu', padding='same')
        self.drop1 = tf.keras.layers.Dropout(0.25)
        self.fc1 = tf.keras.layers.Dense(256, activation='relu')
        self.fc2 = tf.keras.layers.Dense(10, activation=None)
        self.max_pool = tf.keras.layers.MaxPooling2D((2, 2), (2, 2))
        self.flatten = tf.keras.layers.Flatten()
        
    def call(self, inp):

        out = self.conv1(inp)
        out = self.max_pool(out)
        out = self.conv2(out)
        out = self.max_pool(out)
        out = self.conv3(out)
        out = self.max_pool(out)
        out = self.flatten(out)
        out = self.fc1(out)
        out = self.drop1(out)
        out = self.fc2(out)
        
        return out
    
model = Model()

In [9]:
optimizer = tf.keras.optimizers.Adam(learning_rate)
writer = tf.summary.create_file_writer('logs/Adam')

optimizer = tf.keras.optimizers.RMSprop(learning_rate, decay=1e-6)
writer = tf.summary.create_file_writer('logs/RMSprop')

In [10]:
def loss(logits, labels):
    return tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=logits, labels=labels))

def accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1, output_type=tf.int32)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), dtype=tf.float32))

In [11]:
%%time

for iteration, (images, labels) in enumerate(train_ds):
    
    # Forward
    with tf.GradientTape() as tape:
        logits = model(images)
        loss_value = loss(logits, labels)
        
    # Backward
    grads = tape.gradient(loss_value, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    
    # Calc and display loss/accuracy
    if iteration % 2000 == 0:
        test_logits = model(x_test[:256, ...])
        accuracy_value = accuracy(test_logits, y_test[:256, ...])
        
        print("[%4d] Accuracy: %5.2f %%" % (
            iteration, accuracy_value.numpy()*100))
        
        with writer.as_default():
            tf.summary.scalar('accuracy', accuracy_value, iteration)
            tf.summary.scalar('loss', loss_value, iteration)

[   0] Accuracy:  9.38 %
[2000] Accuracy: 59.38 %
[4000] Accuracy: 68.75 %
[6000] Accuracy: 71.88 %
[8000] Accuracy: 70.31 %
[10000] Accuracy: 67.97 %
[12000] Accuracy: 71.48 %
[14000] Accuracy: 70.70 %
Wall time: 2min 9s


In [12]:
%%time

#Оценка модели

test_logits = model(x_test)
accuracy_value = accuracy(test_logits, y_test).numpy()
print("Final Accuracy: %5.2f %%" % (accuracy_value * 100))

Final Accuracy: 70.12 %
Wall time: 718 ms


In [13]:
%load_ext tensorboard
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 5432), started 0:44:26 ago. (Use '!kill 5432' to kill it.)

In [14]:
%reload_ext tensorboard